In [1]:
import pandas as pd 
import numpy as np
import os 
import sys

In [2]:
data = pd.read_csv('/Users/HPhuc/Practice/12. classification/chest-X-rays/output/csv/train.csv', index_col=0)
data

,class_name,class_id,rad_id,x_min,y_min,x_max,y_max,width,height
image_id,,,,,,,,,
50a418190bc3fb1ef1633bf9678929b3,No finding,14,R11,NaN,NaN,NaN,NaN,2332,2580
21a10246a5ec7af151081d0cd6d65dc9,No finding,14,R7,NaN,NaN,NaN,NaN,2954,3159
9a5094b2563a1ef3ff50dc5c7ff71345,Cardiomegaly,3,R10,691.0,1375.0,1653.0,1831.0,2080,2336
051132a778e61a86eb147c7c6f564dfe,Aortic enlargement,0,R10,1264.0,743.0,1611.0,1019.0,2304,2880
063319de25ce7edb9b1c6b8881290140,No finding,14,R10,NaN,NaN,NaN,NaN,2540,3072
...,...,...,...,...,...,...,...,...,...
936fd5cff1c058d39817a08f58b72cae,No finding,14,R1,NaN,NaN,NaN,NaN,2444,3200
ca7e72954550eeb610fe22bf0244b7fa,No finding,14,R1,NaN,NaN,NaN,NaN,1994,2430
aa17d5312a0fb4a2939436abca7f9579,No finding,14,R8,NaN,NaN,NaN,NaN,2048,2500


In [ ]:
classes = np.load('/Users/HPhuc/Practice/12. classification/vinbigdata/output/columns_14.npy')
classes

In [ ]:
concurr_arr = np.zeros([len(classes), len(classes)])

for i, c1 in enumerate(classes):
  for j, c2 in enumerate(classes):
    concurr_arr[i, j] = len(data[(data[c1] == 1) & (data[c2] == 1)])

concurr_mtrix = pd.DataFrame(concurr_arr, columns=classes, index=classes)
concurr_mtrix

In [ ]:
probs_mtrix = concurr_mtrix / np.diag(concurr_mtrix)[:, np.newaxis]
probs_mtrix

In [ ]:
smooth_corr = (probs_mtrix > 0.45).astype(int)  # threshold = 0.45
smooth_corr

Over_smoothing problem

In [ ]:
p = 0.2
reweight = smooth_corr - np.identity(len(classes))
 # smooth_adj = probs_mtrix * p / (probs_mtrix.sum(axis=0) + 1e-6) / (1-p)
reweight = reweight * 0.25 / (reweight.sum(axis=0) + 1e-6)
reweight = reweight + np.identity(len(classes))
reweight

Normalize

In [ ]:
from pathlib import Path 
import parameter as para

# D = np.diag(reweight.sum(axis=1) ** (-0.5))
# reweight = D @ reweight.values.T @ D
# np.save(Path(f'{para.output_path}' + '/correlations_14.npy'), reweight)
# pd.DataFrame(reweight, columns=classes, index=classes)

D = np.diag((reweight.sum(axis=1)) ** (-0.5))
reweight = ((reweight @ D).T) @ D
reweight

In [ ]:
import parameter as para 
from pathlib import Path
import os 

np.save(Path(f'{para.output_path}' + '/correlations_14'), reweight)